In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df = spark.read.format("delta")\
    .load("/Volumes/workspace/bronze/bronzevolume/bookings/data/")

df.display()

In [0]:
df = df.withColumn('amount', col("amount").cast("double"))\
    .withColumn("modified_date", current_timestamp())\
    .withColumn("booking_date", to_date("booking_date"))\
    .drop("_rescued_data")

df.display()

In [0]:
import dlt

@dlt.table(
  name="Stage_bookings"
)
def stage_bookings():
    df = spark.readStream.format("delta")\
        .load('/Volumes/workspace/bronze/bronzevolume/bookings/data/')
    return df


@dlt.view(
    name="trans_bookings"
)
def trans_bookings():
    df = spark.readStream.table("stage_bookings")
    df = df.withColumn('amount', col("amount").cast("double"))\
        .withColumn("modified_date", current_timestamp())\
        .withColumn("booking_date", to_date("booking_date"))\
        .drop("_rescued_data")
    
    return df

rules = {
    "rule1":"booking_id IS NOT NULL",
    "rule2":"passenger_id IS NOT NULL"
}

@dlt.table(
    name='silver_bookings'
)
@dlt.except_all_or_drop(rules)
def silver_bookings():
    df = spark.readStream.table("trans_bookings")
    #or we can use the below one as well
    # df =dlt.read("trans_bookings")
    return df

In [0]:
%sql
SELECT * FROM workspace.silver.silver_bookings;

In [0]:
%sql
SELECT * FROM workspace.silver.stage_bookings

In [0]:
%sql
SELECT * FROM DELTA.`/Volumes/workspace/bronze/bronzevolume/airports/data/`;

In [0]:
%sql
SELECT * FROM workspace.silver.silver_airports

In [0]:
%sql
SELECT * FROM workspace.silver.silver_bookings

In [0]:
%sql
SELECT * FROM workspace.silver.silver_flights

In [0]:
%sql
SELECT * FROM workspace.silver.silver_passengers

In [0]:
%sql
SELECT * FROM DELTA.`/Volumes/workspace/bronze/bronzevolume/bookings/data/`

In [0]:
%sql
select * from delta.`/Volumes/workspace/bronze/bronzevolume/customers/data/`

In [0]:
%sql
select * from delta.`/Volumes/workspace/bronze/bronzevolume/flights/data/`

In [0]:
%sql
select * from workspace.silver.silver_passengers

In [0]:
%sql
select * from workspace.silver.silver_airports

In [0]:
%sql
select * from workspace.silver.silver_bussiness_view

In [0]:
%sql
select * from workspace.silver.silver_flights

In [0]:
%sql
select * from workspace.silver.silver_passengers

In [0]:
%sql
select * from workspace.silver.stage_bookings